## Modelo de Cox. Parte A

Materia: Estadistica Aplicada 

Grupo: 042
                                                                                     
Matricula : 1946646

Por Ana Sofia Saucedo Garcia

### 1 - Importar librerias

In [2]:
#Librerias basicas

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline 

#Libreria especializada

from lifelines import CoxPHFitter, ExponentialFitter, WeibullFitter, LogNormalFitter

# Libreria que requiere mi computadora para importar un archivo de internet
import ssl



In [3]:
# Paso extra para importar el archivo de internet 
ssl._create_default_https_context = ssl._create_unverified_context

### 2 - Importar base de datos

In [4]:
# Datos 
desercion_laboral = pd.read_csv('https://raw.githubusercontent.com/jimmyzac/Estadistica-Aplicada-FCFM-UANL/main/bases_datos/desercion_laboral.csv')

In [5]:
desercion_laboral

,satisfaccion,evaluacion,proyectos,promedio_horas_mes,semestre,accidente,renunciar,ascenso_5years,departamento,salario
0,3.8,5.3,2,157,3,0,1,0,ventas,bajo
1,8.0,8.6,5,262,6,0,1,0,ventas,medio
2,1.1,8.8,7,272,4,0,1,0,ventas,medio
3,7.2,8.7,5,223,5,0,1,0,ventas,bajo
4,3.7,5.2,2,159,3,0,1,0,ventas,bajo
...,...,...,...,...,...,...,...,...,...,...
14994,4.0,5.7,2,151,3,0,1,0,soporte,bajo
14995,3.7,4.8,2,160,3,0,1,0,soporte,bajo
14996,3.7,5.3,2,143,3,0,1,0,soporte,bajo
14997,1.1,9.6,6,280,4,0,1,0,soporte,bajo


### 3 - Depurar la base de datos y estadísticas descriptivas

#### a) Verificar que no haya missing values, en caso de haberlos decidir imputar o eliminar

In [6]:
# Verificar que no haya missing values, en caso de haberlos decidir imputar o eliminar

# contar los valores perdidos / faltantes (missing)
desercion_laboral.isnull().sum()

satisfaccion           0
evaluacion             0
proyectos              0
promedio_horas_mes     0
semestre               0
accidente              0
renunciar              0
ascenso_5years         0
departamento          26
salario                0
dtype: int64

In [7]:
# Imputar datos (promedio)

# Eliminar las observaciones con valores perdidos 
desercion_laboral= desercion_laboral.dropna()


In [8]:
desercion_laboral

,satisfaccion,evaluacion,proyectos,promedio_horas_mes,semestre,accidente,renunciar,ascenso_5years,departamento,salario
0,3.8,5.3,2,157,3,0,1,0,ventas,bajo
1,8.0,8.6,5,262,6,0,1,0,ventas,medio
2,1.1,8.8,7,272,4,0,1,0,ventas,medio
3,7.2,8.7,5,223,5,0,1,0,ventas,bajo
4,3.7,5.2,2,159,3,0,1,0,ventas,bajo
...,...,...,...,...,...,...,...,...,...,...
14994,4.0,5.7,2,151,3,0,1,0,soporte,bajo
14995,3.7,4.8,2,160,3,0,1,0,soporte,bajo
14996,3.7,5.3,2,143,3,0,1,0,soporte,bajo
14997,1.1,9.6,6,280,4,0,1,0,soporte,bajo


#### b) Verificar que no haya valores duplicados con dataframe.duplicated().sum() en caso de haber valores duplicados eliminarlos con dataframe=dataframe.drop_duplicates(keep='first').reset_index(drop=True)

In [9]:
desercion_laboral.duplicated().sum()

2982

In [10]:
# eliminar los valores duplicados 

desercion_laboral=desercion_laboral.drop_duplicates(keep='first').reset_index(drop=True)
desercion_laboral

,satisfaccion,evaluacion,proyectos,promedio_horas_mes,semestre,accidente,renunciar,ascenso_5years,departamento,salario
0,3.8,5.3,2,157,3,0,1,0,ventas,bajo
1,8.0,8.6,5,262,6,0,1,0,ventas,medio
2,1.1,8.8,7,272,4,0,1,0,ventas,medio
3,7.2,8.7,5,223,5,0,1,0,ventas,bajo
4,3.7,5.2,2,159,3,0,1,0,ventas,bajo
...,...,...,...,...,...,...,...,...,...,...
11986,3.7,4.7,2,152,3,0,1,0,desarrollo de producto,bajo
11987,8.4,9.9,4,267,5,0,1,0,desarrollo de producto,bajo
11988,4.1,4.6,2,151,3,0,1,0,desarrollo de producto,bajo
11989,7.6,9.2,4,239,5,0,1,0,desarrollo de producto,bajo


#### c) Verificar que todas las variables sean numéricas, si no son numéricas hacerlas numéricas, en el caso de las categóricas convertirlas en dummies y conservar n-1 dummies

In [11]:
# verificar que las variables son numericas

desercion_laboral.dtypes

satisfaccion          float64
evaluacion            float64
proyectos               int64
promedio_horas_mes      int64
semestre                int64
accidente               int64
renunciar               int64
ascenso_5years          int64
departamento           object
salario                object
dtype: object

No todas las variables son numericas

In [12]:
# Convertimos en dummies las variables categoricas 

desercion_dummies =pd.get_dummies(desercion_laboral,columns=['salario','departamento'])
desercion_dummies= desercion_dummies.drop(['departamento_Investigación','salario_alto'],axis=1)


Nota 
¿Cual dummie se debe de dejar afuera? 
La que elijamos , solo que a la hora de interpretar el modelo la comparacion es con la dummie que se quedo afuera

#### d) Calcular estadísticas descriptivas

In [13]:
# Estadistcas descriptivas 
desercion_dummies.describe()

,satisfaccion,evaluacion,proyectos,promedio_horas_mes,semestre,accidente,renunciar,ascenso_5years,salario_bajo,salario_medio,departamento_administración,departamento_contabilidad,departamento_desarrollo de producto,departamento_informática,departamento_mantenimiento,departamento_mercadotecnia,departamento_recursos humanos,departamento_soporte,departamento_ventas
count,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.00000,11991.000000,11991.000000,11991.000000,11991.000000
mean,6.296581,7.166825,3.802852,200.473522,3.364857,0.154282,0.166041,0.016929,0.478692,0.438746,0.036361,0.051789,0.057210,0.081394,0.18714,0.056125,0.050121,0.151864,0.270119
std,2.410700,1.683426,1.163238,48.727813,1.330240,0.361234,0.372133,0.129012,0.499567,0.496254,0.187194,0.221610,0.232252,0.273451,0.39004,0.230173,0.218204,0.358904,0.444040
min,0.900000,3.600000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,4.800000,5.700000,3.000000,157.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
50%,6.600000,7.200000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
75%,8.200000,8.600000,5.000000,243.000000,4.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1.000000
max,10.000000,10.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000


### 4 - Estimar el modelo de Cox e interpretar cada uno de los coeficientes

In [14]:
# estimar el modelo 
cph1 = CoxPHFitter()
cph1.fit(desercion_dummies,'semestre','renunciar')

<lifelines.CoxPHFitter: fitted with 11991 total observations, 10000 right-censored observations>

In [15]:
cph1.print_summary()

<lifelines.CoxPHFitter: fitted with 11991 total observations, 10000 right-censored observations>
             duration col = 'semestre'
                event col = 'renunciar'
      baseline estimation = breslow
   number of observations = 11991
number of events observed = 1991
   partial log-likelihood = -15884.05
         time fit was run = 2023-09-27 02:01:40 UTC

---
                                      coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                   
satisfaccion                         -0.24       0.78       0.01            -0.26            -0.22                 0.77                 0.80
evaluacion                           -0.01       0.99       0.01            -0.04             0.02                 0.96                 1.02
proyectos                            -0.29       0.75       0.02            -0.33            -0.25                 0.72                 0.78
promedio_horas_mes                    0.00       1.00       0.00             0.00             0.00                 1.00                 1.00
accidente                            -1.25       0.29       0.10            -1.44            -1.05                 0.24                 0.35
ascenso_5years                       -1.38       0.25       0.36            -2.08            -0.68                 0.13                 0.51
salario_bajo                          1.54       4.66       0.15             1.25             1.83                 3.48                 6.23
salario_medio                         1.15       3.15       0.15             0.85             1.44                 2.35                 4.22
departamento_administración           0.22       1.25       0.18            -0.13             0.57                 0.88                 1.76
departamento_contabilidad             0.24       1.27       0.14            -0.04             0.52                 0.96                 1.69
departamento_desarrollo de producto   0.36       1.43       0.14             0.08             0.64                 1.08                 1.90
departamento_informática              0.29       1.34       0.13             0.03             0.56                 1.03                 1.74
departamento_mantenimiento            0.43       1.54       0.12             0.20             0.67                 1.22                 1.95
departamento_mercadotecnia            0.34       1.40       0.14             0.06             0.62                 1.06                 1.86
departamento_recursos humanos         0.43       1.54       0.14             0.15             0.72                 1.16                 2.05
departamento_soporte                  0.45       1.57       0.12             0.21             0.69                 1.23                 2.00
departamento_ventas                   0.35       1.42       0.12             0.12             0.58                 1.13                 1.78

                                      cmp to      z      p   -log2(p)
covariate                                                            
satisfaccion                            0.00 -26.89 <0.005     526.58
evaluacion                              0.00  -0.76   0.44       1.17
proyectos                               0.00 -14.75 <0.005     161.15
promedio_horas_mes                      0.00   3.88 <0.005      13.25
accidente                               0.00 -12.41 <0.005     115.04
ascenso_5years                          0.00  -3.88 <0.005      13.21
salario_bajo                            0.00  10.36 <0.005      81.15
salario_medio                           0.00   7.67 <0.005      45.68
departamento_administración             0.00   1.24   0.21       2.22
departamento_contabilidad               0.00   1.66   0.10       3.36
departamento_desarrollo de producto     0.00   2.49   0.01       6.29
departamento_informática            

In [16]:
# Coeficientes sin redondear
coef= cph1.params_


In [17]:
# exp(coef) sin redondear 
exp_coef=cph1.hazard_ratios_

#### Interpretación de cada uno de los coeficientes 

Notas .
 
 Rechazamos $H_0$ si p-value < $alpha$
 
 Cuando coef(-) , entonces exp(coef)< 1 ,es decir , la variable influye de manera negativa (disminuye) ; (1-exp(coef))

  Cuando coef(+) , entonces exp(coef) > 1 ,es decir , la variable influye de manera positiva (aumenta) ; (exp(coef)-1)




Variable satisfacción 

Rechazo $H_0$ : Su p-value es < 0.005 , esto nos dice que satisfacción tiene efecto significativo en que sus empleados renuncien , entonces si esto se mide en una escala de 0 a 10 ,entre mas satisfecho se encuentra la persona con su trabajo el riesgo de que el empleado renuncie disminuye 22% 


Variable evaluación 

No Rechazo $H_0$ : Su p-value es de 0.44 , esto nos dice que evaluación no tiene efecto significativo en que sus empleados renuncien


Variable proyectos 

Rechazo $H_0$ : Su p-value es < 0.005 , esto nos dice que proyectos tiene efecto significativo en que sus empleados renuncien , entonces por cada proyecto asignado al empleado el riesgo de que el empleado renuncie disminuye 25% 


Variable promedio_horas_mes 

Rechazo $H_0$ : Su p-value es < 0.005 , esto nos dice que el promedio de horas por mes trabajadas tiene efecto significativo en que sus empleados renuncien , aumentado el riesgo por cada hora en 0.184% 

Variable accidente

Rechazo $H_0$ : Su p-value es < 0.005 , esto nos dice que accidente tiene efecto significativo en que sus empleados renuncien , disminuyendo el riesgo si ha tenido un accidente dentro de la empresa 71% con respecto a los que no han tenido accidentes dentro de la empresa

Variable ascenso_5years

Rechazo $H_0$ : Su p-value es < 0.005 , esto nos dice que si ha tenido ascenso durante los ultimos 5 años tiene efecto significativo en que sus empleados renuncien , disminuyendo el riesgo de que un empleado renuncie si ha tenido ascenso durante los ultimos 5 años en un 75% con respecto a los que no han sido ascendidos durante los últimos 5 años

Variable salario_bajo

Rechazo $H_0$ : Su p-value es < 0.005 , esto nos dice que el salario bajo tiene efecto significativo en que sus empleados renuncien , el riesgo de que un empleado con salario bajo renuncie aumenta con respecto a un empleado con salario alto 

Variable salario_medio

Rechazo $H_0$ : Su p-value es < 0.005 , esto nos dice que el salario bajo tiene efecto significativo en que sus empleados renuncien , el riesgo de que un empleado con salario medio renuncie aumenta con respecto a un empleado con salario alto 

Variable departamento_administración

No Rechazo $H_0$ : Su p-value es de 0.21 , esto nos dice que el departamento de administracion no tiene efecto significativo 

Variable departamento_contabilidad

No Rechazo $H_0$ : Su p-value es de 0.10 , esto nos dice que el departamento de contablidad no tiene efecto significativo 

Variable departamento_desarrollo de producto

Rechazo $H_0$ : Su p-value es de 0.01 , esto nos dice que el departamento de desarrollo de producto tiene efecto significativo en que sus empleados renuncien , el riesgo de que un empleado renuncie en el departamento de desarrollo de producto aumenta 43% con respecto a un empleado en el departamento de Investigacion

Variable departamento_informatica 

Rechazo $H_0$ : Su p-value es de 0.03 , esto nos dice que el departamento de informatica tiene efecto significativo en que sus empleados renuncien , el riesgo de que un empleado renunice en el departamento de informatica aumenta 34% con respecto a un empleado en el departamento de Investigacion

Variable departamento_mantenimiento 

Rechazo $H_0$ : Su p-value es de < 0.005 , esto nos dice que el departamento de mantenimiento tiene efecto significativo en que sus empleados renuncien , el riesgo de que un empleado renunice en el departamento de mantenimiento aumenta 54% con respecto a un empleado en el departamento de Investigacion

Variable departamento_mercadotecnia

Rechazo $H_0$ : Su p-value es de 0.02 , esto nos dice que el departamento de mercadotecnia tiene efecto significativo en que sus empleados renuncien , el riesgo de que un empleado renunice en el departamento de mercadotecnia aumenta 40% con respecto a un empleado en el departamento de Investigacion

Variable departamento_recursos humanos 

Rechazo $H_0$ : Su p-value es de < 0.005 , esto nos dice que el departamento de recursos humanos tiene efecto significativo en que sus empleados renuncien , el riesgo de que un empleado renunice en el departamento de recursos humanos aumenta 54% con respecto a un empleado en el departamento de Investigacion

Variable departamento_soporte 

Rechazo $H_0$ : Su p-value es de < 0.005 , esto nos dice que el departamento de soporte tiene efecto significativo en que sus empleados renuncien , el riesgo de que un empleado renunice en el departamento de soporte aumenta 57% con respecto a un empleado en el departamento de Investigacion

Variable departamento_ventas

Rechazo $H_0$ : Su p-value es de < 0.005 , esto nos dice que el departamento de ventas tiene efecto significativo en que sus empleados renuncien , el riesgo de que un empleado renunice en el departamento de ventas aumenta 42% con respecto a un empleado en el departamento de Investigacion

### 5 - Con base en la interpretación de los coeficientes,
 ¿qué recomendación podría hacerle a la empresa para que disminuya la cantidad de renuncias de los empleados?

Analizando la interpretacion de los coeficientes podemos recomendar a la empresa que si un empleado esta satisfecho con con su trabajo , si tiene proyectos asignados , si ha sido ascendido durante los ultimos 5 años y si tiene un salario alto , el riesgo de que un empleado renuncie disminuye 

### 6 - Pruebe si el modelo de Cox cumple el supuesto de riesgos proporcionales, de no hacerlo estime el modelo AFT adecuado

El modelo de Cox esta construido sobre el supuesto de riesgo proporcional 

Tenemos que verificar que el supuesto se cumpla 

In [18]:
cph1.check_assumptions(desercion_dummies,p_value_threshold=0.05)


# H_0: Se cumple riesgo proporcional constante 
# H_a: No Se cumple riesgo proporcional constante 

The ``p_value_threshold`` is set at 0.05. Even under the null hypothesis of no violations, some
covariates will be below the threshold by chance. This is compounded when there are many covariates.
Similarly, when there are lots of observations, even minor deviances from the proportional hazard
assumption will be flagged.

With that in mind, it's best to use a combination of statistical tests and visual tests to determine
the most serious violations. Produce visual plots using ``check_assumptions(..., show_plots=True)``
and looking for non-constant lines. See link [A] below for a full example.



<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 11991 total observations, 10000 right-censored observations>
         test_name = proportional_hazard_test

---
                                          test_statistic      p  -log2(p)
accidente                           km              0.03   0.87      0.21
                                    rank            1.14   0.29      1.81
ascenso_5years                      km              3.66   0.06      4.17
                                    rank            6.71   0.01      6.71
departamento_administración         km              5.08   0.02      5.37
                                    rank            5.74   0.02      5.92
departamento_contabilidad           km              0.09   0.77      0.39
                                    rank            1.14   0.29      1.81
departamento_desarrollo de producto km              1.03   0.31      1.69
                                    rank            0.85   0.36      1.49
departamento_informática            km              0.48   0.49      1.03
                                    rank            0.53   0.47      1.10
departamento_mantenimiento          km              0.47   0.49      1.02
                                    rank            2.26   0.13      2.91
departamento_mercadotecnia          km              2.66   0.10      3.28
                                    rank            6.33   0.01      6.39
departamento_recursos humanos       km              0.68   0.41      1.29
                                    rank            2.86   0.09      3.46
departamento_soporte                km              0.00   0.98      0.03
                                    rank            0.89   0.35      1.53
departamento_ventas                 km              1.04   0.31      1.70
                                    rank            4.36   0.04      4.76
evaluacion                          km            443.05 <0.005    324.32
                                    rank          427.15 <0.005    312.82
promedio_horas_mes                  km            270.84 <0.005    199.74
                                    rank          307.41 <0.005    226.21
proyectos                           km            574.13 <0.005    419.05
                                    rank          684.93 <0.005    499.11
salario_bajo                        km              0.37   0.54      0.88
                                    rank            0.00   0.96      0.06
salario_medio                       km              0.16   0.69      0.53
                                    rank            0.10   0.75      0.42
satisfaccion                        km           1257.74 <0.005    912.74
                                    rank          845.24 <0.005    614.90



1. Variable 'satisfaccion' failed the non-proportional test: p-value is <5e-05.

   Advice 1: the functional form of the variable 'satisfaccion' might be incorrect. That is, there
may be non-linear terms missing. The proportional hazard test used is very sensitive to incorrect
functional forms. See documentation in link [D] below on how to specify a functional form.

   Advice 2: try binning the variable 'satisfaccion' using pd.cut, and then specify it in
`strata=['satisfaccion', ...]` in the call in `.fit`. See documentation in link [B] below.

   Advice 3: try adding an interaction term with your time variable. See documentation in link [C]
below.


2. Variable 'evaluacion' failed the non-proportional test: p-value is <5e-05.

   Advice 1: the functional form of the variable 'evaluacion' might be incorrect. That is, there may
be non-linear terms missing. The proportional hazard test used is very sensitive to incorrect
functional forms. See documentation in link [D] below on how to 

[]

Si alguna variable no cumple el supuesto , entonces no se puede utilizar el modelo Cox 

Por lo que no se puede utilizar el modelo Cox

#### Estime el modelo AFT adecuado

Para usar el modelo de riesgo acelerado primero tenemos que verificar si es Weibull , exponencial o Lognormal. Y luego estimar el modelo AFT 

In [19]:
mexpo = ExponentialFitter().fit(desercion_dummies['semestre'],desercion_dummies['renunciar'])
mweibull = WeibullFitter().fit(desercion_dummies['semestre'],desercion_dummies['renunciar'])
mlogn = LogNormalFitter().fit(desercion_dummies['semestre'],desercion_dummies['renunciar'])

In [20]:
mexpo.AIC_, mweibull.AIC_ , mlogn.AIC_

(15965.45892201228, 12511.512867661068, 11587.92169210356)

In [21]:
# Dado el valor del AIC significa que debe ser un modelo AFT Lognormal

from lifelines import LogNormalAFTFitter
lognAFT = LogNormalAFTFitter().fit(desercion_dummies,'semestre','renunciar')
lognAFT.print_summary()


<lifelines.LogNormalAFTFitter: fitted with 11991 total observations, 10000 right-censored observations>
             duration col = 'semestre'
                event col = 'renunciar'
   number of observations = 11991
number of events observed = 1991
           log-likelihood = -4876.57
         time fit was run = 2023-09-27 02:01:44 UTC

---
                                             coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
param  covariate                                                                                                                                   
mu_    accidente                             0.26       1.30       0.02             0.22             0.30                 1.25                 1.35
       ascenso_5years                        0.37       1.44       0.06             0.24             0.49                 1.27                 1.63
       departamento_administración           0.05       1.05       0.04            -0.03             0.13                 0.97                 1.13
       departamento_contabilidad            -0.04       0.96       0.03            -0.11             0.02                 0.90                 1.02
       departamento_desarrollo de producto  -0.05       0.95       0.03            -0.12             0.01                 0.89                 1.01
       departamento_informática             -0.05       0.95       0.03            -0.11             0.01                 0.90                 1.02
       departamento_mantenimiento           -0.08       0.93       0.03            -0.13            -0.02                 0.88                 0.98
       departamento_mercadotecnia           -0.04       0.97       0.03            -0.10             0.03                 0.90                 1.03
       departamento_recursos humanos        -0.09       0.92       0.03            -0.15            -0.02                 0.86                 0.98
       departamento_soporte                 -0.08       0.92       0.03            -0.14            -0.03                 0.87                 0.97
       departamento_ventas                  -0.06       0.94       0.03            -0.11            -0.00                 0.90                 1.00
       evaluacion                            0.01       1.01       0.00            -0.00             0.01                 1.00                 1.01
       promedio_horas_mes                   -0.00       1.00       0.00            -0.00            -0.00                 1.00                 1.00
       proyectos                             0.10       1.10       0.01             0.09             0.11                 1.09                 1.11
       salario_bajo                         -0.37       0.69       0.03            -0.43            -0.31                 0.65                 0.73
       salario_medio                        -0.29       0.75       0.03            -0.34            -0.23                 0.71                 0.80
       satisfaccion                          0.07       1.07       0.00             0.06             0.07                 1.06                 1.07
       Intercept                             1.34       3.83       0.04             1.25             1.43                 3.50                 4.18
sigma_ Intercept                            -1.11       0.33       0.02            -1.14            -1.08                 0.32                 0.34

                                             cmp to      z      p   -log2(p)
param  covariate                                                            
mu_    accidente                               0.00  12.98 <0.005     125.59
       ascenso_5years                          0.00   5.80 <0.005      27.19
       departamento_administración             0.00   1.21   0.23       2.14
       departamento_contabilidad               0.00  -1.27   0.20       2.30
       departamento_desarrollo de producto     0.00  -1.54   0.12       3.01
       d

### 7 - Interprete los coeficientes del modelo AFT


Variable satisfacción 

Rechazo $H_0$ : Su p-value es < 0.005 , esto nos dice que satisfacción tiene efecto significativo en que sus empleados renuncien , entonces si esto se mide en una escala de 0 a 10 ,entre mas satisfecho se encuentra la persona con su trabajo , aumenta la mediana del tiempo de supervivencia un 7% 


Variable evaluación 

No Rechazo $H_0$ : Su p-value es de 0.14 , esto nos dice que evaluación no tiene efecto significativo en que sus empleados renuncien


Variable proyectos 

Rechazo $H_0$ : Su p-value es < 0.005 , esto nos dice que proyectos tiene efecto significativo en que sus empleados renuncien , entonces por cada proyecto asignado al empleado aumenta la mediana del tiempo de supervivencia un 10% 


Variable promedio_horas_mes 

Rechazo $H_0$ : Su p-value es 0.02 , esto nos dice que el promedio de horas por mes trabajadas tiene efecto significativo en que sus empleados renuncien , disminuye la mediana del tiempo de supervivencia por cada hora en un 0.0308% 

Variable accidente

Rechazo $H_0$ : Su p-value es < 0.005 , esto nos dice que accidente tiene efecto significativo en que sus empleados renuncien ,un accidente dentro de la empresa aumenta la mediana del tiempo de supervivencia un 30% con respecto a los que no han tenido accidentes dentro de la empresa

Variable ascenso_5years

Rechazo $H_0$ : Su p-value es < 0.005 , esto nos dice que si ha tenido ascenso durante los ultimos 5 años tiene efecto significativo en que sus empleados renuncien , si un empleado si ha tenido ascenso durante los ultimos 5 años aumenta la mediana del tiempo de supervivencia en un 44% con respecto a los que no han sido ascendidos durante los últimos 5 años

Variable salario_bajo

Rechazo $H_0$ : Su p-value es < 0.005 , esto nos dice que el salario bajo tiene efecto significativo en que sus empleados renuncien , un empleado con salario bajo disminuye la mediana del tiempo de supervivencia un 31% con respecto a un empleado con salario alto 

Variable salario_medio

Rechazo $H_0$ : Su p-value es < 0.005 , esto nos dice que el salario bajo tiene efecto significativo en que sus empleados renuncien , un empleado con salario medio disminuye la mediana del tiempo de supervivencia un 25% con respecto a un empleado con salario alto 

Variable departamento_administración

No Rechazo $H_0$ : Su p-value es de 0.23 , esto nos dice que el departamento de administracion no tiene efecto significativo 

Variable departamento_contabilidad

No Rechazo $H_0$ : Su p-value es de 0.20 , esto nos dice que el departamento de contablidad no tiene efecto significativo 

Variable departamento_desarrollo de producto

No Rechazo $H_0$ : Su p-value es de 0.12 , esto nos dice que el departamento de desarrollo de producto no tiene efecto significativo 

Variable departamento_informatica

No Rechazo $H_0$ : Su p-value es de 0.14 , esto nos dice que el departamento de informatica no tiene efecto significativo 

Variable departamento_mantenimiento

Rechazo $H_0$ : Su p-value es de 0.01 , esto nos dice que el departamento de mantenimiento tiene efecto significativo , el departamento de mantenmiento disminuye la mediana del tiempo de supervivencia un 7% con respecto al departamento de investigacion

Variable departamento_mercadotecnia

No Rechazo $H_0$ : Su p-value es de 0.14 , esto nos dice que el departamento de mercadotecnia no tiene efecto significativo 

Variable departamento_recursos humanos

Rechazo $H_0$ : Su p-value es de 0.01 , esto nos dice que el departamento de recursos humanos tiene efecto significativo , el departamento de recursos humanos disminuye la mediana del tiempo de supervivencia un 8% con respecto al departamento de investigacion

Variable departamento_soporte

Rechazo $H_0$ : Su p-value es <0.005 , esto nos dice que el departamento de soporte tiene efecto significativo , el departamento de soporte disminuye la mediana del tiempo de supervivencia un 8% con respecto al departamento de investigacion

Variable departamento_ventas

Rechazo $H_0$ : Su p-value es de 0.03 , esto nos dice que el departamento de ventas tiene efecto significativo , el departamento de ventas disminuye la mediana del tiempo de supervivencia un 6% con respecto al departamento de investigacion